In [9]:

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from skimage import feature
from PIL import ImageOps
import os
import numpy as np
from PIL import Image, ExifTags
from sklearn.cluster import KMeans
from skimage import feature, morphology


In [10]:




def apply_kmeans(image_np, n_clusters=2):
    pixels = image_np.reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(pixels)
    labels = kmeans.labels_
    mask = labels.reshape(image_np.shape[:2])
    
    leaf_cluster = np.argmin([np.linalg.norm(center - [0, 255, 0]) for center in kmeans.cluster_centers_])
    binary_mask = (mask == leaf_cluster).astype(np.uint8)
    
    binary_mask = morphology.remove_small_objects(binary_mask.astype(bool), min_size=500).astype(np.uint8)
    
    return binary_mask

def apply_lbp(image_np, radius=1, n_points=8):
    gray_image = Image.fromarray(image_np).convert('L')
    gray_np = np.array(gray_image)
    lbp = feature.local_binary_pattern(gray_np, n_points, radius, method="uniform")
    return (lbp > np.percentile(lbp, 75)).astype(np.uint8)

def exif_transpose(img):
    if not hasattr(img, '_getexif') or img._getexif() is None:
        return img

    exif = img._getexif()
    orientation = exif.get(0x0112, None) 

    transformations = {
        2: Image.FLIP_LEFT_RIGHT,
        3: Image.ROTATE_180,
        4: Image.FLIP_TOP_BOTTOM,
        5: Image.FLIP_LEFT_RIGHT,
        6: Image.ROTATE_270,
        7: Image.FLIP_LEFT_RIGHT,
        8: Image.ROTATE_90
    }

    return img.transpose(transformations.get(orientation, None)) if orientation in transformations else img

def resize_and_pad(img, target_size=(256, 256)):
    """Resizes the image maintaining its aspect ratio and pads the result to a target size."""
    bbox = ImageOps.crop(img, border=0).getbbox()
    if not bbox:
        return Image.new('RGB', target_size, (0, 0, 0))  # Return black image if no content
    
    cropped = img.crop(bbox)
    ratio = min(target_size[0] / cropped.width, target_size[1] / cropped.height)
    new_size = (int(cropped.width * ratio), int(cropped.height * ratio))
    
    # Resize while maintaining aspect ratio
    img_resized = cropped.resize(new_size, Image.LANCZOS)
    
    # Pad to the target size
    delta_w = target_size[0] - img_resized.width
    delta_h = target_size[1] - img_resized.height
    padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))
    return ImageOps.expand(img_resized, padding, fill=(0, 0, 0))

def prepare_image_for_prediction(image_path, save_dir):
    try:
        image = Image.open(image_path)
        image = exif_transpose(image)
        image_np = np.array(image)

        kmeans_mask = apply_kmeans(image_np)
        lbp_mask = apply_lbp(image_np)
        combined_mask = np.logical_or(kmeans_mask, lbp_mask).astype(np.uint8)

        combined_mask = morphology.opening(combined_mask, morphology.disk(3))
        combined_mask = morphology.closing(combined_mask, morphology.disk(3))

        segmented_color_image = image_np * np.stack([combined_mask]*3, axis=2)

        if hasattr(image, "_getexif") and image._getexif() is not None:
            exif_data = {ExifTags.TAGS[k]: v for k, v in image._getexif().items() if k in ExifTags.TAGS and isinstance(v, (bytes, str))}
            exif_bytes = ExifTags.dump(exif_data)
        else:
            exif_bytes = None
        
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f'prepared_{os.path.basename(image_path)}')
        segmented_img = Image.fromarray(segmented_color_image)

        if exif_bytes:
            segmented_img.save(save_path, exif=exif_bytes)
        else:
            segmented_img.save(save_path)

        # Resize and pad the segmented image to 2016x2016
        with Image.open(save_path) as img:
            new_img = resize_and_pad(img)
            new_img.save(save_path)

        return save_path

    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None








# Example Usage
uploaded_image_path = "D:\\BscSE\\Final Project\\HerbQuest\\software\\herbquest_backend\\prediction\\pred.jpg"
save_segmented_dir = "D:\\BscSE\\Final Project\\HerbQuest\\software\\herbquest_backend\\prediction\\"

prepared_image_path = prepare_image_for_prediction(uploaded_image_path, save_segmented_dir)


C:\Users\Anushan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [11]:
# Load the pre-trained model
model = load_model('D:\BscSE\Final Project\HerbQuest\software\herbquest_backend\model\herbquest_model_256.h5')

In [13]:

from keras.preprocessing.image import img_to_array, load_img

def get_prediction(model, prepared_image_path):

    # Load the pre-trained model
    # model = load_model('D:\BscSE\Final Project\HerbQuest\software\herbquest_backend\model\herbquest_model_256.h5')

    # uploaded_image_path = "D:\\BscSE\\Final Project\\HerbQuest\\software\\herbquest_backend\\prediction\\pred.jpg"
    # save_segmented_dir = "D:\\BscSE\\Final Project\\HerbQuest\\software\\herbquest_backend\\prediction\\"
    # prepared_image_path = prepare_image_for_prediction(uploaded_image_path, save_segmented_dir)

    # Load the image
    img = load_img(prepared_image_path, target_size=(256, 256))
    
    # Convert the PIL Image to a numpy array
    img_array = img_to_array(img)
    
    # Normalize the image (this step may or may not be necessary based on your model training)
    img_array = img_array / 255.0
    
    # Expand dimensions to match the shape the model expects
    img_array = np.expand_dims(img_array, axis=0)
    
    # Get the model's prediction
    predictions = model.predict(img_array)

    # Get the highest-probability class label
    predicted_class = np.argmax(predictions[0])
    predicted_confidence = np.max(predictions[0])

    # Map the class label to the class name
    class_names = [
        'ButterflyPea',
        'CommonWireweed',
        'CrownFlower',
        'GreenChireta',
        'HeartLeavedMoonseed',
        'HolyBasil',
        'IndianCopperLeaf',
        'IndianJujube',
        'IndianStingingNettle',
        'IvyGourd',
        'RosaryPea',
        'SmallWaterClover',
        'SpiderWisp',
        'SquareStalkedVine',
        'TrellisVine'
    ]

    predicted_class_name = class_names[predicted_class]

    # Display the prediction probabilities for each class
    print("Prediction Probabilities:")
    for class_name, predicted_probability in zip(class_names, predictions[0]):
        print(f"{class_name}: {predicted_probability * 100:.2f}%")


    return predicted_class

# Use the function
predicted_class = get_prediction(model, prepared_image_path)
print(f"The predicted class is: {predicted_class}")



1/1 [==============================] - 0s 60ms/step
Prediction Probabilities:
ButterflyPea: 0.00%
CommonWireweed: 0.00%
CrownFlower: 0.00%
GreenChireta: 0.00%
HeartLeavedMoonseed: 0.00%
HolyBasil: 0.00%
IndianCopperLeaf: 0.00%
IndianJujube: 0.00%
IndianStingingNettle: 0.00%
IvyGourd: 0.28%
RosaryPea: 0.01%
SmallWaterClover: 0.00%
SpiderWisp: 99.71%
SquareStalkedVine: 0.00%
TrellisVine: 0.00%
The predicted class is: 12


In [ ]:
prepared_image = prepare_image(image_path)